In [50]:
import mido
import os
import string
import matplotlib.pyplot as plt
import numpy as np

from torch.utils.data import Dataset, DataLoader
import torch
from torch import nn

In [51]:
class MidiTranscriber():
    def __init__(self):
        pass
    


    def Sequence(self, track):
        result = []
        notes = {}
        time = 0
        final_result = []
        data = [] 
        state = [0] * 128
        tick_counter = 0
        
        tick_interval = 20
        
        for i in range(16):
            data.append(state)
            
        for msg in track:
            time += msg.time
            tick_counter += msg.time
            
            if(tick_counter > tick_interval):
                final_result.append(data)
                tick_counter = 0
            
            if msg.type == 'note_on':
                if msg.velocity != 0:
                    past = notes.get(msg.note)
                    if(past == None):
                        past = []
                    notes[msg.note] = past + [time , 1] 
                    result.append([time, msg.note, msg.velocity,  msg.channel, msg.type])
                    data[msg.channel][msg.note] = msg.velocity
                    
                else:
                    pass
            if msg.type == 'note_off':
                past = notes.get(msg.note)
                if(past == None):
                    past = []
                notes[msg.note] = past + [time , 0]
                result.append([time ,msg.note, msg.velocity ,  msg.channel, msg.type])
                data[msg.channel][msg.note] = 0
                
        return result, data, final_result
        
        
    def BuildArray(self,midi):
        Lenghth = [len(tr) for tr in midi.tracks]
        track_length = max(Lenghth)
        min_track =  track_length/10 #Only accept tracks 1/10 of length.
        all_arys = []
        training = []
        for track in range(len(midi.tracks)):
            out = []
            valid_track = (len(midi.tracks[track]) > min_track)
            if(valid_track):
                track = midi.tracks[track]
                res, data, final_result = self.Sequence(track)
                
                #print(len(final_result))
                
                
                all_arys.append(res)
                training.append(data)
            
        #sums = all_arys.sum(axis=1)
        #ends = np.where(sums > 0)[0]
        return all_arys, data, final_result

        builder = []
        
        

In [52]:
scriber = MidiTranscriber()

In [65]:
def Parser(composer, path):
    midis = os.listdir(path)
    
    pair = []
    
    for file in midis:
        if(file == ".DS_Store"):
            continue 
        midis_path = path + "/" + file
        #print(midis_path)
        
        midi = mido.MidiFile(midis_path)
        #print(len(midi.tracks[0]) + len(midi.tracks[1]) + len(midi.tracks[2]))
        #print(len(midi.tracks[0]))
        arr, data, final_result = scriber.BuildArray(midi)
        
        
        sequence_size = 100
        
        div = len(final_result) / sequence_size
        last = 0
        for i in range(int(div)):

            clip = final_result[last:sequence_size*int((i+1))]
            last = int(sequence_size)*(i+1)
            
            x = clip
            y= composer
 
            pair.append([x,y])
    
            if(not len(x) == 100):
                print(len(x))
        
    return arr, data, final_result, pair
        


In [70]:
dataset = "Composer_Dataset/NN_midi_files_extended/train"
composers = ['bach', 'bartok', 'byrd', 'chopin', 'handel', 'hummel', 'mendelssohn', 'mozart', 'schumann']

pairs = []

for composer in composers:
    path = dataset +"/"+ composer
    print(composer)
    arr, data, final_result,pair = Parser(composer, path)
    #print(len(pair))
    pairs.append(pair)
    
    
    



bach
bartok
byrd
chopin
handel
hummel
mendelssohn
mozart
schumann


In [71]:
data = np.array(data)
data.shape

(16, 128)

In [72]:
final_result = np.array(final_result)

In [73]:
final_result.shape

(1736, 16, 128)

In [74]:
pair = []
for i in pairs:
    for j in i:
        pair.append(j)

In [75]:
len(pair)

5598

In [76]:
test = np.array(pair)

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 2 dimensions. The detected shape was (5598, 2) + inhomogeneous part.

In [97]:
class Dataset(Dataset):
     
    def enum(self,y):
        composers = ["bach", "bartok", "byrd", "chopin", "handel", "hummel", "mendelssohn","mozart", "schumann"]
        #print(y)
        for i in range(len(composers)):
            if composers[i] == y:
                slot = i
        out = [0] * 9
        out[slot] = 1
        
        return out
        
    def __init__(self, pair):
        self.pair = pair
    def __len__(self):
        return len(self.pair)
    
    def __getitem__(self,idx):
        
        x = torch.tensor(self.pair[idx][0])
        y = torch.tensor(self.enum(self.pair[idx][1]))
        return x,y
        

In [98]:
train_data = Dataset(pair)

In [103]:
train_data[200][0].shape

torch.Size([100, 16, 128])

In [ ]:
for i in range(100):
    train

In [189]:
train_dataloader = DataLoader(train_data,batch_size=3,shuffle=True)

In [190]:
len(train_dataloader)

123